### First attempt at building a Neural Network to learn a non-linear F(s)


In [20]:
import torch
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F

print(torch.__version__)

1.9.0


In [21]:
#Preprocess Data
df = pd.read_csv('../bricks_data/dataset_geometric.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268046 entries, 0 to 268045
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   lrg_density        268046 non-null  float64
 1   elg_density        268046 non-null  float64
 2   qso_density        268046 non-null  float64
 3   stellar_density    268046 non-null  float64
 4   airmass_galaxy     268046 non-null  float64
 5   fwhm_galaxy        268046 non-null  float64
 6   ebv_galaxy         268046 non-null  float64
 7   ccdnphotom_galaxy  268046 non-null  float64
 8   ccdskysb_galaxy_g  268046 non-null  float64
 9   ccdskysb_galaxy_r  268046 non-null  float64
 10  ccdskysb_galaxy_z  268046 non-null  float64
 11  exptime_galaxy_g   268046 non-null  float64
 12  exptime_galaxy_r   268046 non-null  float64
 13  exptime_galaxy_z   268046 non-null  float64
 14  meansky_galaxy_g   268046 non-null  float64
 15  meansky_galaxy_r   268046 non-null  float64
 16  me

### Defining The Dataset Class Inheriting from Torch.dataset to be able to use a dataloader for training

In [43]:
class DensitySurvey(Dataset):
    def __init__(self, df, galaxy_type):
        self.data = df[0:1000]
        # Extracting Targets and Input
        if galaxy_type == "LRG":
            self.target = self.data['lrg_density'].to_numpy(copy=True)
        if galaxy_type == "ELG":
            self.target = self.data['elg_density'].to_numpy(copy=True)
        if galaxy_type == "QSO":
            self.target = self.data['qso_density'].to_numpy(copy=True)
        self.input = self.data.drop(columns=['lrg_density','elg_density','qso_density']).to_numpy(copy=True)

        # Scaling
        scaler = preprocessing.MinMaxScaler()
        self.input = scaler.fit_transform(self.input)
        self.target = scaler.fit_transform(self.target.reshape(-1, 1))
        print(self.input.shape)
        print(self.target.shape)


    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return torch.from_numpy(self.input[idx]), torch.tensor(self.target[idx])


In [44]:
df = pd.read_csv('../bricks_data/dataset_geometric.csv')
train_df, test_df = train_test_split(df, test_size=0.33, random_state=44, shuffle=True)
traindata = DensitySurvey(train_df, 'LRG')
testdata = DensitySurvey(test_df, 'LRG')

print(traindata.__len__())
print(testdata.__len__())


(1000, 17)
(1000, 1)
(1000, 17)
(1000, 1)
1000
1000


### Define Model and Hyperparameters



In [ ]:
class Net(nn.Module):
    def __init__(self, n_feature = 17, n_hidden = 10, n_output = 1):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_feature,n_hidden)
        #self.fc2 = nn.Linear(n_hidden,n_hidden)
        self.predict = nn.Linear(n_hidden,n_output)

    def forward(self,x):
        out = F.relu(self.fc1(x))
        out = self.predict(out)
        return out

device = 'cpu'

model = Net().to(device)

 

In [ ]:
# Defining Loss
criterion = nn.MSELoss()

#Defining Hyperparemeters
no_epochs = 500 #very low, but computational power not sufficient for more iterations
batch = 1024
learning_rate = 0.001

#Using the Adam Method for Stochastic Optimisation
optimiser = optim.Adam(model.parameters(), lr=learning_rate)